In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib
import matplotlib.font_manager as fm
from sklearn.metrics import f1_score
## 맥북용 한글
# mpl.rcParams['font.family'] = 'AppleGothic'
# mpl.rcParams['axes.unicode_minus'] = False   # 마이너스(-) 부호 깨짐 방지

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

# 사용할 한글 폰트 경로 (나눔고딕 또는 맑은고딕)
font_path = 'C:\\Windows\\Fonts\\malgun.ttf'  #

# 폰트 파일이 있는지 확인 후 적용
if os.path.exists(font_path):
    fontprop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = fontprop.get_name()
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 깨짐 방지
else:
    print("폰트 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 테스트
plt.plot([1, 2, 3], [1, 2, 3])
plt.title("한글 테스트")
plt.show()

In [ ]:
df = pd.read_csv('../data/dataset.csv')
df.head()

In [ ]:
df.info()
# 데이터 개수는 4424

In [ ]:
df.describe()

In [ ]:
df.isna().sum()
# 컬럼에 결측치 확인

In [ ]:
# 불필요한 컬럼(열) 삭제
df = df.drop(['Application mode'], axis=1)
df = df.drop(['Application order'], axis=1)
df = df.drop(['Nacionality'], axis=1)
df = df.drop(['Mother\'s qualification'], axis=1)
df = df.drop(['Father\'s qualification'], axis=1)
df = df.drop(['International'], axis=1)
df = df.drop(['Curricular units 1st sem (credited)'], axis=1)
df = df.drop(['Curricular units 1st sem (enrolled)'], axis=1)
df = df.drop(['Curricular units 1st sem (evaluations)'], axis=1)
df = df.drop(['Curricular units 1st sem (without evaluations)'], axis=1)
df = df.drop(['Curricular units 2nd sem (credited)'], axis=1)
df = df.drop(['Curricular units 2nd sem (enrolled)'], axis=1)
df = df.drop(['Curricular units 2nd sem (evaluations)'], axis=1)
df = df.drop(['Curricular units 2nd sem (without evaluations)'], axis=1)
df = df.drop(['Unemployment rate'], axis=1)
df = df.drop(['Inflation rate'], axis=1)
df = df.drop(['GDP'], axis=1)

df.head()


In [ ]:
# target에 범주를 매핑(데이터 설명에 나와있는 범주 A,B,C -> 0,1,2)
targetvariable = {'Dropout': 0,'Graduate': 1,'Enrolled': 2}
df['Target'] = df['Target'].map(targetvariable)

In [ ]:
df.rename(columns={
    'Marital status': '결혼 상태',
    'Application mode': '지원 방법',
    'Application order': '지원 순서',
    'Course': '수강 과정',
    'Daytime/evening attendance': '주/야간 수업',
    'Previous qualification': '이전 학력',
    'Nationality': '국적',
    "Mother's qualification": '어머니 학력',
    "Father's qualification": '아버지 학력',
    "Mother's occupation": '어머니 직업',
    "Father's occupation": '아버지 직업',
    'Displaced': '이재민 여부',
    'Educational special needs': '특수 교육 필요 여부',
    'Debtor': '연체 여부',
    'Tuition fees up to date': '등록금 납부 여부',
    'Gender': '성별',
    'Scholarship holder': '장학금 수혜 여부',
    'Age': '입학 나이',
    'International': '국제 학생 여부',
    'Curricular units 1st sem (credited)': '1학기 학점 인정 과목 수',
    'Curricular units 1st sem (enrolled)': '1학기 수강 과목 수',
    'Curricular units 1st sem (evaluations)': '1학기 평가된 과목 수',
    'Curricular units 1st sem (approved)': '1학기 합격 과목 수',
    'Curricular units 1st sem (grade)': '1학기 성적 평균',
    'Curricular units 1st sem (without evaluations)': '1학기 평가 미진행 과목 수',
    'Curricular units 2nd sem (credited)': '2학기 학점 인정 과목 수',
    'Curricular units 2nd sem (enrolled)': '2학기 수강 과목 수',
    'Curricular units 2nd sem (evaluations)': '2학기 평가된 과목 수',
    'Curricular units 2nd sem (approved)': '2학기 합격 과목 수',
    'Curricular units 2nd sem (grade)': '2학기 성적 평균',
    'Curricular units 2nd sem (without evaluations)': '2학기 평가 미진행 과목 수',
    'Unemployment rate': '실업률',
    'Inflation rate': '인플레이션율',
    'GDP': 'GDP',
    'Target': '학업 성취도 (타겟)'
}, inplace=True)


In [ ]:
corr_mat = df.corr(numeric_only=True)
corr_mat

In [ ]:
plt.figure(figsize=(16, 16))
sns.heatmap(corr_mat, annot=True, cmap='coolwarm', fmt='.2f', annot_kws={'size': 8})

plt.tight_layout()
plt.xticks(rotation=24)
# plt.savefig('heatmap_pre.png', dpi=200)
plt.show()

# 데이터 분할

In [ ]:
!pip install catboost
!pip install xgboost
!pip install lightgbm

In [ ]:
df = df[df['Target'] != 2]

df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings("ignore", category=UserWarning)


# 범주형 변수 목록 (사용자 정의)
cat_cols = [
    'Marital status',
    'Course',
    'Daytime/evening attendance',
    'Previous qualification',
    "Mother's occupation",
    "Father's occupation",
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder'
]

# 수치형 변수 목록 (cat_cols 제외한 나머지)

num_cols = [
    'Age',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)'
]


# 전처리 파이프라인 정의
numeric_transformer = Pipeline([('scaler', StandardScaler())])

categorical_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore',sparse_output=False))])

preprocessor = ColumnTransformer([('num', numeric_transformer, num_cols),('cat', categorical_transformer, cat_cols)]).set_output(transform='pandas')

# 파이프라인 정의 (scaler 대신 preprocessor)
pipeline = Pipeline([('preprocessor', preprocessor),('classifier', LogisticRegression())])


param_grid = [
  # LogisticRegression
  {
        'classifier': [LogisticRegression(max_iter=1000, random_state=42)],
        'classifier__C': [0.01, 0.1, 1, 10],
        'classifier__solver': ['liblinear', 'lbfgs'],
    },
  # SVC
  {
        'classifier': [SVC(random_state=42)],
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__C': [0.1, 1, 10],
    },
  # KNeighborsClassifier
  {
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [3, 5, 7],
        'classifier__weights': ['uniform', 'distance'],
    },

  # RandomForestClassifier (전처리 그대로 둬도 무방, 스케일링 영향 적음)
  {
        'classifier': [RandomForestClassifier(random_state=42)],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [None, 10, 20],
    },
  # XGBClassifier
  {
        'classifier': [XGBClassifier(eval_metric='logloss', random_state=42, n_jobs=-1)],
        'classifier__n_estimators': [100, 200, 300, 500],
        'classifier__max_depth': [3, 5, 7, 9],
        'classifier__learning_rate': [0.05, 0.1, 0.2],
     },
  # LGBMClassifier
  {
        'classifier': [LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1, feature_name='auto')],
        'classifier__n_estimators': [100, 200, 300, 400],
        'classifier__max_depth': [-1, 5, 10, 10],
        'classifier__num_leaves': [20, 31, 40, 50],
        'classifier__learning_rate': [0.01, 0.05, 0.1],
        'classifier__subsample': [0.7, 0.8, 0.9, 1.0],
        'classifier__colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'classifier__reg_alpha': [0, 0.1, 0.01],
        'classifier__reg_lambda': [0, 0.1, 0.01]
    },
  # CatBoostClassifier
  {
        'classifier': [CatBoostClassifier(verbose=0, random_state=42)],
        'classifier__iterations': [50, 100, 200, 500],
        'classifier__depth': [4, 6, 8 ,10],
        'classifier__learning_rate': [0.01, 0.05,  0.1],
    }


]


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    cv=cv,
    n_jobs=-1,
    verbose=2,
    scoring='f1'
)

grid_search.fit(X_train, y_train)

print(f"최적 검증점수: {grid_search.best_score_:.4f}")
print(f"최적 파라미터: {grid_search.best_params_}")

val_score = grid_search.score(X_val, y_val)
print(f"Validation Accuracy with best params: {val_score:.4f}")


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import pandas as pd

# 1. grid_search 결과 DataFrame 변환 및 모델별 최고 점수/파라미터 출력
results_df = pd.DataFrame(grid_search.cv_results_)
results_df['classifier_name'] = results_df['param_classifier'].apply(lambda x: type(x).__name__)

print("===== 모델별 최고 CV 점수 및 하이퍼파라미터 =====\n")
for clf_name, group in results_df.groupby('classifier_name'):
    idx = group['mean_test_score'].idxmax()
    best_row = group.loc[idx]
    print(f"모델: {clf_name}")
    print(f"  최고 검증점수: {best_row['mean_test_score']:.4f}")
    print(f"  최고 파라미터: {best_row['params']}\n")

# 2. 최적 모델로 X_val 예측 (항상 파이프라인 전체에 원본 X_val을 넣어야 함)
best_model = grid_search.best_estimator_

# predict_proba 지원 여부 확인
if hasattr(best_model.named_steps['classifier'], 'predict_proba'):
    probs = best_model.predict_proba(X_val)[:, 1]
    preds = (probs >= 0.5).astype(int)
else:
    # SVC 등 일부 모델은 predict_proba 미지원, predict 사용
    preds = best_model.predict(X_val)
    # 만약 SVC(kernel='linear') 등 이진분류에서 decision_function이 있으면
    # threshold 적용이 가능하지만, 여기서는 predict 결과 사용

# 3. 평가 지표 출력
accuracy = accuracy_score(y_val, preds)
f1 = f1_score(y_val, preds)

print(f"\n[Best Model 평가 결과]")
print(f"Validation Accuracy with threshold 0.5: {accuracy:.4f}")
print(f"Validation F1 Score with threshold 0.5: {f1:.4f}")

In [ ]:
import joblib

# grid_search.best_estimator_는 최적의 파이프라인 전체(전처리+모델)를 포함합니다.
joblib.dump(grid_search.best_estimator_, 'best_model_pipeline.pkl')

In [ ]:
# 저장한 모델 불러오기
loaded_model = joblib.load('best_model_pipeline.pkl')

# 바로 예측 가능 (전처리 자동 적용)
y_pred = loaded_model.predict(X_test)
y_pred

In [ ]:
best_model = grid_search.best_estimator_

test_score = best_model.score(X_val, y_val)
print(f"Test Accuracy: {test_score:.4f}")

## 딥러닝

In [ ]:
deep_df = df.copy()

In [ ]:
deep_df = deep_df [deep_df ['Target'] != 2].copy()

In [ ]:
deep_df['Target'].value_counts()

In [ ]:
# 범주형 변수 목록 (사용자 정의)
cat_cols = [
    'Marital status',
    'Course',
    'Daytime/evening attendance',
    'Previous qualification',
    "Mother's occupation",
    "Father's occupation",
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder'
]

# 수치형 변수 리스트
num_cols = [col for col in deep_df.columns if col not in cat_cols + ['Target']]

In [ ]:
scaler = StandardScaler()
deep_df[num_cols] = scaler.fit_transform(deep_df[num_cols])

In [ ]:
X = deep_df.drop(columns=['Target']).values
y = deep_df['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class StudentDropoutDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # CrossEntropyLoss를 위해 long 사용

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = StudentDropoutDataset(X_tr, y_tr)       # 실제 학습용(train)
val_dataset =  StudentDropoutDataset(X_val, y_val)      # 검증용(validation)
test_dataset =  StudentDropoutDataset(X_test, y_test)   # 테스트용(test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 이진분류(0,1)니까 2 출력

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class DropoutNet(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.2):
        super(DropoutNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [ ]:
def train_by_scheduler(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=5):
    for epoch in range(epochs):
        # === 학습 ===
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            batch_size = X_batch.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += batch_size

        train_loss = running_loss / total
        train_accuracy = correct / total

        # === 검증 ===
        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                batch_size = X_batch.size(0)
                val_running_loss += loss.item() * batch_size
                preds = outputs.argmax(dim=1)
                val_correct += (preds == y_batch).sum().item()
                val_total += batch_size

        val_loss = val_running_loss / val_total
        val_accuracy = val_correct / val_total

        # 학습률 스케줄러 업데이트
        scheduler.step()

        lr = scheduler.get_last_lr()[0]

        print(f"Epoch {epoch + 1}/{epochs} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f} | LR: {lr:.6f}")

In [ ]:
def train_with_early_stopping(
    model, train_loader, val_loader, criterion, optimizer, scheduler, device,
    epochs=100, patience=10, min_delta=1e-4
):
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        # === 학습 ===
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            batch_size = X_batch.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += batch_size

        train_loss = running_loss / total
        train_accuracy = correct / total

        # === 검증 ===
        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                batch_size = X_batch.size(0)
                val_running_loss += loss.item() * batch_size
                preds = outputs.argmax(dim=1)
                val_correct += (preds == y_batch).sum().item()
                val_total += batch_size

        val_loss = val_running_loss / val_total
        val_accuracy = val_correct / val_total

        scheduler.step()
        lr = scheduler.get_last_lr()[0]

        print(f"Epoch {epoch + 1}/{epochs} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f} | LR: {lr:.6f}")

        # === Early Stopping 체크 ===
        if best_val_loss - val_loss > min_delta:
            best_val_loss = val_loss
            patience_counter = 0
            # 필요하다면 모델 가중치 저장 가능
            # torch.save(model.state_dict(), 'best_model.pt')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DropoutNet(input_dim=X_tr.shape[1], dropout_rate=0.3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

train_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=100,         # 최대 에폭
    patience=10,        # 개선 없으면 10번 후 종료
    min_delta=1e-4      # 개선 기준
)